In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance
import random
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

In [2]:
objects = ['r', 'h', 'c', 's', 'e']

coordinates = {'r': (0,5,2),
              'h': (4,4,1),
              'c': (2,0,0),
               's': (4,3,2),
               'e': (4,1,1),
              'start': (4,5,2),
              'table': (2,2,2)}

c1 = {'r': 1.2,
    'h': 1.0,
    'c': 1.2,
    's': 1.2,
    'e': 1.2}

k1 = {'r': 1.0,
    'h': 0.9,
    'c': 0.95,
    's': 1.0,
    'e': 1.0}

c0 = {'r': 1.0,
    'h': 1.0,
    'c': 1.0,
    's': 1.0,
    'e': 1.0}

k0 = {'r': 1.0,
    'h': 1.0,
    'c': 1.0,
    's': 1.0,
    'e': 1.0}

start_coordinates = [[4,5,2],[2,2,2],[2,2,2],[0,5,2],[2,2,2]]

In [3]:
def predict_sequence(possible_items, coordinates, c, k, start_coordinates, dimensions=[3,]):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    
    if dimensions[0] == 3: # no changes if 3D
        pass
        
    elif dimensions[0] == 2: # 2D: remove obsolete coordinate
        if dimensions[1] == 'xy':
            coords = {key: value[:-1] for key, value in coords.items()}
            start_coords = [x[:-1] for x in start_coords]
                
        elif dimensions[1] == 'xz':
            #coords = {key: value for key, value in coords.items()}
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                coords[key] = new_value

            for x in range (0, len(start_coords)):
                start_coords[x] = [start_coords[x][0], start_coords[x][-1]]
        
        elif dimensions[1] == 'yz':
            coords = {key: value[1:] for key, value in coords.items()}
            start_coords = [x[1:] for x in start_coords]
                
    elif dimensions[0] == 1: # 1D: choose appropriate coordinate
        if dimensions[1] == 'x':
            coords = {key: value[0] for key, value in coords.items()}
            start_coords = [x[0] for x in start_coords]
        
        elif dimensions[1] == 'y':
            coords = {key: value[1] for key, value in coords.items()}
            start_coords = [x[1] for x in start_coords]
        
        elif dimensions[1] == 'z':
            coords = {key: value[2] for key, value in coords.items()}
            start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                start_coords[coord_index], 
                                coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [7]:
predict_sequence(objects,coordinates, c1, k1, start_coordinates, dimensions=[3])

['h', 's', 'e', 'r', 'c']

In [11]:
edit_list = []

for x in range(0,1000):
    result = ''.join(predict_sequence(objects,coordinates, c0, k0, start_coordinates, dimensions=[3,'xyz']))
    #print(result)
    dl = damerau_levenshtein_distance('rhcse',result)
    edit_list.append(dl)
    
avg = np.mean(edit_list)
avg

4.0